# Problem logarytmu dyskretnego

Niech $(G,\circ)$ będzie grupą z działaniem $\circ$ i elementem neutralnym $1_G$. Wtedy dla dowolnego elementu $a\in G$ i $k\in\mathbb{Z}$ definiujemy *potęgę* $$a^k =\left\{\begin{array}{cc}
\underbrace{a\circ a\circ \ldots \circ a}_{k}&\text{ dla }k>0\\
1_G&\text{ dla }k=0\\
\underbrace{a^{-1}\circ a^{-1}\circ \ldots \circ a^{-1}}_{k}&\text{ dla }k<0
\end{array}\right.$$
gdzie $a^{-1}$ jest elementem odwrotnym do $a$.

Dla $a,b\in G$, $b\neq 1_G$, **logarytmem dyskretnym** $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a$.

## Logarytm dyskretny w $\mathbb{Z}_n$

W przypadku pierścienia $\mathbb{Z}_n$ logarytmem dyskretnym $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a\bmod n$, o ile w ogóle istnieje.

Specyficzną sytuacją w teorii liczb jest gdy $n=p$ jest liczbą pierwszą a $q$ jest **pierwiastkiem pierwotnym** $\mod p$. Wtedy:
- potęgi $q^k\mod p$ generują cały zbiór $[1,p-1]$, tzn. $q$ jest tzw. **generatorem grupy multiplikatywnej** rzędu $p-1$
- logarytm dyskretny $\log_q a$ istnieje dla każdego niezerowego elementu $a\in \mathbb{Z}_p$

### Skąd wziąć generator grupy multiplikatywnej?

,,Pierwiastek pierwotny (nazywany także pierwiastkiem prymitywnym) to element grupy multiplikatywnej modulo n, którego rząd jest równy rzędowi grupy. W teorii grup oznacza to, że jest to generator grupy – element, który poprzez kolejne potęgowanie generuje wszystkie elementy danej grupy (za Wikipedia).''

W ogólnym przypadku i dla dużych $p$ znalezienie generatora grupy jest obliczeniowo trudne. Najbardziej naiwne postępowanie polega na wylosowaniu liczby $g$ i sprawdzeniu czy podnosząc ją do potęgi ($k\in \mathbb{Z}$) uzyskamy wszystkie elementy grupy. 

Mniej naiwne postępowanie opiera się na własności rzędu elementu w grupie:

1. Rozkład $p-1$ na czynniki pierwsze (aby znaleźć liczby względnie pierwsze z $p$):
$$ p-1 = \prod q_i^{e_i} $$

2. Sprawdzamy liczby z zakresu $g \in \{2,..., p-1\}$ czy spełniają kongruencję:
$$ g^\frac{p-1}{q_i} \equiv 1 \bmod p $$ dla wszystkich czynników $q_i$
    
    a. jeśli przynajmniej raz kongruencja jest spełniona $g$ nie jest generatorem!
    
    b. w przeciwnym wypadku $g$ jest generatorem

## Zadanie 1
Napisz kod znajdujący generatory grupy multiplikatywnej modulo liczba pierwsza $p$ stosując opisaną powyżej metodę. Możesz zastosować metodę odwrotną: wylosuj liczby pierwsze, które będą czynnikami pierwszymi $p-1$, upewnij się jednak czy $p$ jest liczbą pierwszą (!).  

In [1]:
import numpy as np

def find_prime_factors(p):
    if p < 2:
        return list()

    factors = list()
    while p % 2 == 0:
        p //= 2
        factors.append(2)

    for d in range(3, int(np.sqrt(p))+1, 2):
        while p % d == 0:
            p //= d
            factors.append(d)
        if p == 1:
            break

    if p > 1:
        factors.append(p)
    return factors

def find_generator(p):
    prime_factors = set(find_prime_factors(p-1))
    for g in range(2, p):
        flag = True
        for q in prime_factors:
            if pow(g, (p-1)//q, mod=p) == 1:
                flag = False
                break
        if flag:
            return g
    return None
    
p = 2137
g = find_generator(p)
print(f"Generator g={g} for group of modulo p={p}")

# Sprawdź czy rzeczywiście g generuje wszystkie elementy
group = {i for i in range(1, p)}
generated = {pow(g, k, p) for k in range(1, p)}
print("Correct generator" if group == generated else "Incorrect generator")

Generator g=10 for group of modulo p=2137
Correct generator


# Wymiana klucza typu Diffie-Hellman

Alice i Bob uzgadniają klucz publiczny będący liczbą pierwszą $p$ oraz $q$ - pierwiastkiem pierwotnym mod $p$.
- sekret Alice: liczba całkowita $n\in \mathbb{Z}_p\setminus\{0\}$
- sekret Boba: liczba całkowita $m\in \mathbb{Z}_p\setminus\{0\}$
- Alice generuje $x=q^n\mod p$ i wysyła do Boba
- Bob generuje $y=q^m\mod p$ i wysyła Alice
- Alice oblicza klucz $k=y^n\mod p$
- Bob oblicza klucz $k=x^m\mod p$


## Zadanie 2.

Zaimplementuj powyższy algorytm wymiany klucza. Użyj parametrów, które otrzymałeś w Zadaniu 1

In [2]:
secret_A = 69
secret_B = 420

def generate_exchange_value(secret, p, q):
    return pow(q, secret, mod=p)

def calculate_key(secret, received, p):
    return pow(received, secret, mod=p)

value_A = generate_exchange_value(secret_A, p, g)
value_B = generate_exchange_value(secret_B, p, g)

key_A = calculate_key(secret_A, value_B, p)
key_B = calculate_key(secret_B, value_A, p)

print(f"p={p}, q={g}")
print(f"Alice: secret={secret_A}, generated={value_A}, key={key_A}")
print(f"Bob:  secret={secret_B}, generated={value_B}, key={key_B}")
print("Keys match" if key_A == key_B else "Keys don't match")

p=2137, q=10
Alice: secret=69, generated=1589, key=27
Bob:  secret=420, generated=1779, key=27
Keys match


## Algorytm baby-step giant-step

Jeden z najprostszych (poza metodą naiwną) algorytmów poszukiwania logarytmu dyskretnego w grupach cyklicznych.

Niech $p$ będzie liczbą pierwszą oraz niech $q$ będzie pierwiastkiem pierwotnym modulo $p$. Dla niezerowego $a\in\mathbb{Z}_p$ szukamy liczby $k\in\mathbb{Z}$ takiej, że $q^k=a\mod p$

### Krok 1.
- $m=\lceil\sqrt{p-1}\rceil$
- tworzymy pomocniczą tablicę potęg: dla wszystkich $i\in [0,m)$ obliczamy parę $(i,q^i)$
- obliczamy $r=(q^{-1})^m$
### Krok 2.
- $b=a$
- dla wszystkich $j\in [0,m)$:
    - sprawdzamy, czy para $(i,b)$ jest elementem tablicy potęg dla pewnego $i$
    - jeżeli tak, to $k=jm+i$ i kończymy algorytm
    - jeżeli nie, to $b=br$ i kontynuujemy pętlę

## Zadanie 3.

Zaimplementować algorytm baby-step giant-step. Przetestować dla podanych danych testujących.

```Dane testujące:
p = 7
q = 3
a = 4

m = 3
tablica_testowa = [1,3,2]
r = 6
k = 4 (j = 1, i = 1)
```

```
p = 29
q = 8
a = 10

m = 6
tablica_testowa = [1,8,6,19,7,27]
r = 9
k = 17 (j = 2, i = 5)
```

```
p = 113
q = 76
a = 84

m = 11
tablica_testowa = [1,76,13,84,56,75,50,71,85,19,88]
r = 70
k = 3 (j = 0, i = 3)
```

In [3]:
def extended_euclid(a, b):
    r0, r1 = a, b
    s0, s1 = 1, 0
    t0, t1 = 0, 1
    while r1 != 0:
        q = int(r0/r1)
        r0, r1 = r1, r0 - q*r1
        s0, s1 = s1, s0 - q*s1
        t0, t1 = t1, t0 - q*t1
    return r0, s0, t0

def modulo_inverse(a, n):
    d, s, _ = extended_euclid(a, n)
    if d != 1:
        print("d != 1 - no inverse modulo")
        return None
    return s if s >= 0 else s+n

def baby_giant(p, q, a):
    m = int(np.ceil(np.sqrt(p-1)))
    # hash map for faster access, table for tests
    table = [pow(q, i, mod=p) for i in range(m)]
    hash_map = {}
    for i, power in enumerate(table):
        hash_map[power] = i
    r = pow(modulo_inverse(q, p), m, mod=p)

    result = {}
    result['m'] = m
    result['table'] = table
    result['r'] = r

    for j in range(m):
        if a in hash_map:
            i = hash_map[a]
            result['j'] = j
            result['i'] = i
            result['k'] = j*m + i
            return result
        a = (a*r) % p

a = 69
res = baby_giant(p, g, a)
check = pow(g, res['k'], mod=p)
print(f"Input: a={a}, p={p}, q={g}")
print(f"Output: k={res['k']}")
print(f"{g}^{res['k']} = {check} mod {p}")
print("ok" if check == a else "wrong")

Input: a=69, p=2137, q=10
Output: k=971
10^971 = 69 mod 2137
ok


In [4]:
def get_test(p, q, a, m, table, r, k, j, i):
    return {'p': p, 'q': q, 'a': a, 'm': m, 'table': table, 'r': r, 'k': k, 'j': j, 'i': i}

def compare_results(expected, calculated):
    names = ['m', 'table', 'r', 'k', 'j', 'i']
    incorrect = list()
    for name in names:
        if expected[name] != calculated[name]:
            incorrect.append(name)
    return incorrect

def run_tests(tests):
    passed = 0
    for no, test in enumerate(tests):
        print(f"Test {no+1} (p={test['p']}, q={test['q']}, a={test['a']}):")
        result = baby_giant(test['p'], test['q'], test['a'])
        incorrect = compare_results(test, result)
        if len(incorrect) == 0:
            passed += 1
            print("Passed")
        else:
            print("Failed - incorrect results:")
            for inc in incorrect:
                print(f" - {inc}: calculated - {result[inc]}, expected - {test[inc]}")
        print()

    print(f"Passed {passed}/{len(tests)} tests")
    return passed == len(tests)

tests = [
    get_test(7, 3, 4, 3, [1,3,2], 6, 4, 1, 1),
    get_test(29, 8, 10, 6, [1,8,6,19,7,27], 9, 17, 2, 5),
    get_test(113, 76, 84, 11, [1,76,13,84,56,75,50,71,85,19,88], 70, 3, 0, 3)
]

run_tests(tests)

Test 1 (p=7, q=3, a=4):
Passed

Test 2 (p=29, q=8, a=10):
Passed

Test 3 (p=113, q=76, a=84):
Passed

Passed 3/3 tests


True